Given a biased coin of unknown bias, can you generate fair random bits? Given such a coin, can you generate random bits with a desired bias? Given a random source of numbers chosen independently and uniformly from $ [0, 1] $, can you generate numbers according to a normal random variable?

This chapter will focus on answering such questions, providing a range of methods varying in simplicity and efficiency.

One can easily discard this chapter without affecting the understanding the rest of the lecture notes, although some of the ideas first discussed in chapter might be helpful in the following chapters.